In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.fft import fft, fftfreq

In [2]:
# read cata from csv files
data_floor1 = pd.read_csv('raw_data/OnTable.csv')
data_floor2 = pd.read_csv('raw_data/onIpad.csv')
data_floor3_1 = pd.read_csv('raw_data/side1.csv')
data_floor3_2 = pd.read_csv('raw_data/record.csv')

In [3]:
def rotate_accelerometer_to_world_frame(sensor_df):
    acc_data = sensor_df[['acc_x', 'acc_y', 'acc_z']].copy()
    acc_gravity = sensor_df[['acc_gx', 'acc_gy', 'acc_gz']].copy()
    rotation_rate = sensor_df[['gyro_x', 'gyro_y', 'gyro_z']].copy()
    
    columns = ['acc_x', 'acc_y', 'acc_z', 'acc_gx', 'acc_gy', 'acc_gz', 'mean_magnitude']
    result_df = pd.DataFrame(columns=columns)
    for acc, acc_g, gyro in zip(acc_data.itertuples(index=False), acc_gravity.itertuples(index=False), rotation_rate.itertuples(index=False)):
        roll = gyro.gyro_x
        pitch = gyro.gyro_y
        yaw = gyro.gyro_z
        r = R.from_euler('zyx', [yaw, pitch, roll], degrees=False)
    
        acc_world = r.apply([acc.acc_x, acc.acc_y, acc.acc_z])
        gravity_world = r.apply([acc_g.acc_gx, acc_g.acc_gy, acc_g.acc_gz])
        mean_magnitude = np.linalg.norm(acc_world)
        
        result = pd.DataFrame([list(acc_world) + list(gravity_world) + [mean_magnitude]], columns=columns)
        result_df = pd.concat([result_df, result], ignore_index=True)
        
    return result_df
    
    

In [4]:
def compute_frequency_domain(signal, interval):
    N = len(signal)
    fs = 1000 / interval
    frequencies = fftfreq(N, d=1/fs)
    fft_values = np.abs(fft(signal))
    
    pos_mask = frequencies > 0
    frequencies = frequencies[pos_mask]
    fft_values = fft_values[pos_mask]
    
    power = fft_values**2
    return np.sum(frequencies * power) / np.sum(power), frequencies[np.argmax(fft_values)]
    

## Preprocessing function

In [5]:
def csv_preprocess(data, window_size=2000, data_interval=500):
    features = [
        'avg_acc_x', 'median_acc_x', 'std_acc_x', 'min_x', 'max_x', 'mean_abs_x',
        'avg_acc_y', 'median_acc_y', 'std_acc_y', 'min_y', 'max_y', 'mean_abs_y',
        'avg_acc_z', 'median_acc_z', 'std_acc_z', 'min_z', 'max_z', 'mean_abs_z',
        'avg_acc_gx', 'avg_acc_gy', 'avg_acc_gz',
        'mean_magnitude', 'signal_magnitude_area',
        'mean_freq_x', 'dominant_freq_x',
        'mean_freq_y', 'dominant_freq_y',
        'mean_freq_z', 'dominant_freq_z',
        'lat_diff', 'lon_diff',
        'label'
    ]
    result_df = pd.DataFrame(columns=features)
    
    # window sliding approach
    n_row = window_size // data_interval
    for window_start in range(0, len(data) - n_row + 1):
        window_end = window_start + n_row
        this_window = data.iloc[window_start:window_end]
        
        rotated_df = rotate_accelerometer_to_world_frame(this_window)
        
        # accelerometer of this window frame
        mean_acc_x = rotated_df['acc_x'].mean()
        median_acc_x = rotated_df['acc_x'].median()
        std_acc_x = rotated_df['acc_x'].std()
        min_acc_x = rotated_df['acc_x'].min()
        max_acc_x = rotated_df['acc_x'].max()
        mean_abs_x = rotated_df['acc_x'].abs().mean()
        # ------------------------------------
        mean_acc_y = rotated_df['acc_y'].mean()
        median_acc_y = rotated_df['acc_y'].median()
        std_acc_y = rotated_df['acc_y'].std()
        min_acc_y = rotated_df['acc_y'].min()
        max_acc_y = rotated_df['acc_y'].max()
        mean_abs_y = rotated_df['acc_y'].abs().mean()
        # ------------------------------------
        mean_acc_z = rotated_df['acc_z'].mean()
        median_acc_z = rotated_df['acc_z'].median()
        std_acc_z = rotated_df['acc_z'].std()
        min_acc_z = rotated_df['acc_z'].min()
        max_acc_z = rotated_df['acc_z'].max()
        mean_abs_z = rotated_df['acc_z'].abs().mean()
        
        # accelerometer (including gravity) of this window frame
        mean_acc_gx = rotated_df['acc_gx'].mean()
        # std_acc_gx = rotated_df['acc_gx'].std()
        # min_acc_gx = rotated_df['acc_gx'].min()
        # max_acc_gx = rotated_df['acc_gx'].max()
        # ------------------------------------
        mean_acc_gy = rotated_df['acc_gy'].mean()
        # std_acc_gy = rotated_df['acc_gy'].std()
        # min_acc_gy = rotated_df['acc_gy'].min()
        # max_acc_gy = rotated_df['acc_gy'].max()
        # ------------------------------------
        mean_acc_gz = rotated_df['acc_gz'].mean()
        # std_acc_gz = rotated_df['acc_gz'].std()
        # min_acc_gz = rotated_df['acc_gz'].min()
        # max_acc_gz = rotated_df['acc_gz'].max()
        
        # other features
        mean_magnitude = rotated_df['mean_magnitude'].mean()
        # ------------------------------------
        signal_magnitude_area = np.sum(np.abs([mean_acc_x, mean_acc_y, mean_acc_z]))
        # ------------------------------------
        mean_freq_x, dominant_freq_x = compute_frequency_domain(rotated_df['acc_x'], data_interval)
        mean_freq_y, dominant_freq_y = compute_frequency_domain(rotated_df['acc_y'], data_interval)
        mean_freq_z, dominant_freq_z = compute_frequency_domain(rotated_df['acc_z'], data_interval)
        # ------------------------------------
        lats = list(this_window['gps_lat'])
        lons = list(this_window['gps_lon'])
        diff_lat = lats[0] - lats[len(lats) - 1]
        diff_lon = lons[0] - lons[len(lons) - 1]
        
        label_map = {'Halt': 0, 'Forward': 1, 'Turn': 2}
        label = label_map.get(this_window['action'].mode()[0], -1)

        row = pd.DataFrame([[mean_acc_x, median_acc_x, std_acc_x, min_acc_x, max_acc_x, mean_abs_x, 
                             mean_acc_y, median_acc_y, std_acc_y, min_acc_y, max_acc_y, mean_abs_y, 
                             mean_acc_z, median_acc_z, std_acc_z, min_acc_z, max_acc_z, mean_abs_z, 
                             mean_acc_gx, mean_acc_gy, mean_acc_gz, 
                             mean_magnitude, signal_magnitude_area, 
                             mean_freq_x, dominant_freq_x, 
                             mean_freq_y, dominant_freq_y, 
                             mean_freq_z, dominant_freq_z, 
                             diff_lat, diff_lon, 
                             label]], columns=features)
        result_df = pd.concat([result_df, row], ignore_index=True)
        
    return result_df


## Model Evaluation

In [6]:
from sklearn.model_selection import  cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [7]:
preprocessed_data = csv_preprocess(data_floor1)
preprocessed_data = pd.concat([preprocessed_data, csv_preprocess(data_floor2)], ignore_index=True)
preprocessed_data = pd.concat([preprocessed_data, csv_preprocess(data_floor3_1)], ignore_index=True)
preprocessed_data = pd.concat([preprocessed_data, csv_preprocess(data_floor3_2)], ignore_index=True)

/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_36716/1732417339.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, result], ignore_index=True)
/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_36716/2207287629.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, row], ignore_index=True)
/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_36716/1732417339.py:19: FutureWarning: The behavior of DataFra

In [8]:
X = preprocessed_data.drop('label', axis=1)
y = preprocessed_data['label'].astype(int)

In [9]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=69)
scoring = ['accuracy', 'f1_macro', 'f1_weighted']

### Decision Tree Classifier

In [10]:
model = DecisionTreeClassifier(class_weight='balanced', random_state=69)

In [11]:
cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring)

print("Accuracy scores:", cv_results['test_accuracy'])
print("Mean accuracy:", cv_results['test_accuracy'].mean())

print("F1 macro scores:", cv_results['test_f1_macro'])
print("Mean F1 macro:", cv_results['test_f1_macro'].mean())

Accuracy scores: [0.7109375  0.67578125 0.71289062 0.703125   0.63867188]
Mean accuracy: 0.68828125
F1 macro scores: [0.68719234 0.64841487 0.69574602 0.67890117 0.61327457]
Mean F1 macro: 0.6647057942875023


### Random Forest Classifier

In [12]:
model = RandomForestClassifier(class_weight='balanced', random_state=69)

In [13]:
cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring)

print("Accuracy scores:", cv_results['test_accuracy'])
print("Mean accuracy:", cv_results['test_accuracy'].mean())

print("F1 macro scores:", cv_results['test_f1_macro'])
print("Mean F1 macro:", cv_results['test_f1_macro'].mean())

Accuracy scores: [0.77734375 0.77929688 0.73828125 0.77929688 0.72460938]
Mean accuracy: 0.759765625
F1 macro scores: [0.76012332 0.76165556 0.71969591 0.76605375 0.71224456]
Mean F1 macro: 0.7439546194010733


### LightGBM

In [14]:
model = LGBMClassifier(class_weight='balanced', random_state=69)

In [15]:
cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000433 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6115
[LightGBM] [Info] Number of data points in the train set: 2048, number of used features: 25
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000364 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6112
[LightGBM] [Info] Number of data points in the train set: 2048, number of used features: 25
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Auto-choosing row-

In [16]:
print("Accuracy scores:", cv_results['test_accuracy'])
print("Mean accuracy:", cv_results['test_accuracy'].mean())

print("F1 macro scores:", cv_results['test_f1_macro'])
print("Mean F1 macro:", cv_results['test_f1_macro'].mean())

Accuracy scores: [0.8515625  0.84375    0.80273438 0.8359375  0.81445312]
Mean accuracy: 0.8296875
F1 macro scores: [0.8393884  0.83071034 0.79118239 0.82706693 0.80061074]
Mean F1 macro: 0.8177917611518619


## Model Training
We decided to go with LightGBM model since it has high accuracy score and F1 score.

In [17]:
import pickle

In [18]:
model = LGBMClassifier(class_weight='balanced', random_state=69)
model.fit(X, y)

file_path = 'Models/lightGBM-model_v1.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(model, f)
    
print(f'The model has been saved to: { file_path }')
    

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000358 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6222
[LightGBM] [Info] Number of data points in the train set: 2560, number of used features: 25
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
The model has been saved to: Models/lightGBM-model_v1.pkl
